In [25]:
import pandas as pd
import numpy as np
from urllib.parse import quote
import webbrowser
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [26]:
df = pd.read_csv('data/danhmuc_and_sapnhap.csv')

In [27]:
df.shape

(10642, 19)

In [28]:
# Tính lại isDividedWard
df.drop(columns=['isDividedWard'], inplace=True)
count_unit = df.groupby(['province', 'district', 'ward']).size().reset_index(name='count_unit')
count_unit['isDividedWard'] = np.where(count_unit['count_unit'] > 1, True, False)
df = pd.merge(df, count_unit[['province', 'district', 'ward', 'isDividedWard']], on=['province', 'district', 'ward'], how='left')
df['isDividedWard'].fillna(False, inplace=True)

In [34]:
# Các ward bị chia
df_divided = df[df['isDividedWard']==True].copy()
df_divided.sort_values(by=['province', 'district', 'ward', 'newWardAreaKm2'], inplace=True)

In [37]:
df_divided

,provinceCode,isMergedProvince,districtCode,districtType,wardCode,wardType,isMergedWard,province,district,ward,newProvince,newWard,newProvinceCode,newWardCode,newWardType,newWardLat,newWardLon,newWardAreaKm2,isDividedWard
10228,92,True,925,Huyện,31249.0,Xã,True,Thành phố Cần Thơ,Huyện Cờ Đỏ,Xã Thạnh Phú,Thành phố Cần Thơ,Nhu Gia,33,3234.0,xã,9.47039,105.855,75.59,True
10229,92,True,925,Huyện,31249.0,Xã,False,Thành phố Cần Thơ,Huyện Cờ Đỏ,Xã Thạnh Phú,Thành phố Cần Thơ,Thạnh Phú,33,3257.0,xã,10.12950,105.405,99.07,True
10247,92,True,927,Huyện,31268.0,Xã,True,Thành phố Cần Thơ,Huyện Thới Lai,Xã Tân Thạnh,Thành phố Cần Thơ,Trường Thành,33,3275.0,xã,10.04230,105.610,59.09,True
10246,92,True,927,Huyện,31268.0,Xã,True,Thành phố Cần Thơ,Huyện Thới Lai,Xã Tân Thạnh,Thành phố Cần Thơ,Tân Thạnh,33,3254.0,xã,9.62301,106.064,70.80,True
10223,92,True,924,Huyện,31246.0,Xã,True,Thành phố Cần Thơ,Huyện Vĩnh Thạnh,Xã Thạnh Quới,Thành phố Cần Thơ,Gia Hòa,33,3204.0,xã,9.43175,105.800,77.06,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9829,87,True,874,Huyện,30157.0,Xã,True,Tỉnh Đồng Tháp,Huyện Thanh Bình,Xã Tân Thạnh,Tỉnh Đồng Tháp,Thanh Bình,31,3069.0,xã,10.41230,106.386,86.00,True
9737,87,True,867,Thành phố,29902.0,Phường,True,Tỉnh Đồng Tháp,Thành phố Sa Đéc,Phường 3,Tỉnh Đồng Tháp,Sa Đéc,31,3046.0,phường,10.32900,105.739,46.90,True
9738,87,True,867,Thành phố,29902.0,Phường,True,Tỉnh Đồng Tháp,Thành phố Sa Đéc,Phường 3,Tỉnh Đồng Tháp,Cao Lãnh,31,2997.0,phường,10.43990,105.624,73.30,True
9740,87,True,867,Thành phố,29908.0,Phường,True,Tỉnh Đồng Tháp,Thành phố Sa Đéc,Phường 4,Tỉnh Đồng Tháp,Sa Đéc,31,3046.0,phường,10.32900,105.739,46.90,True


In [38]:
# Các district có ward bị chia
df_divided_district = df_divided[['province', 'district']].drop_duplicates()

In [39]:
# # Tìm tên đường phố thủ công
# for index, row in tqdm(df_divided_district.iterrows(), total=df_divided_district.shape[0]):
#     keyword = row['district'] + ', ' + row['province'] + ' wikipedia'
#     search_url = f"https://www.google.com/search?q={quote(keyword)}"
#     webbrowser.open(search_url)
#     wikipedia = input("wikipedia:").upper()
#     if wikipedia != 'NO':
#         element = input("element:")
#     else:
#         element = 'NO'
#     df_divided_district.loc[index, 'wikipedia'] = wikipedia
#     df_divided_district.loc[index, 'element'] = element
# df_divided_district.to_csv('data/df_divided_district.csv', index=False)

In [40]:
df_divided_district = pd.read_csv('data/df_divided_district.csv')

# Không có street, default

In [41]:
from vietunits.utils import geolocator, check_point_in_polygon, find_nearest_point

In [42]:
location = geolocator.geocode('Phường 15, Tân Bình, Hồ Chí Minh')

In [46]:
df[(df['district']=='Quận Tân Bình') & (df['ward']=='Phường 15')]

,provinceCode,isMergedProvince,districtCode,districtType,wardCode,wardType,isMergedWard,province,district,ward,newProvince,newWard,newProvinceCode,newWardCode,newWardType,newWardLat,newWardLon,newWardAreaKm2,isDividedWard
8750,79,True,766,Quận,27007.0,Phường,True,Thành phố Hồ Chí Minh,Quận Tân Bình,Phường 15,Thành phố Hồ Chí Minh,Tân Bình,29,2807.0,phường,10.8006,106.642,2.12,True
8751,79,True,766,Quận,27007.0,Phường,True,Thành phố Hồ Chí Minh,Quận Tân Bình,Phường 15,Thành phố Hồ Chí Minh,Tân Sơn,29,2819.0,phường,10.8224,106.650,10.12,True


In [50]:
df_divided_ward = df_divided[['province', 'district', 'ward']].drop_duplicates()

In [52]:
for index, row in tqdm(df_divided_ward.iterrows(), total=len(df_divided_ward)):
    address = row['ward'] + ', ' + row['district'] + ', ' + row['province']
    location = geolocator.geocode(address)
    df_divided_ward.loc[index, ['wardLat', 'wardLon', 'wardAddress']] = location.latitude, location.longitude, location.address
df_divided_ward.to_csv('data/df_divided_ward.csv', index=False)

  1%|          | 4/511 [00:02<04:38,  1.82it/s]


KeyboardInterrupt: 

In [48]:
df_divided_ward = pd.read_csv('data/df_divided_ward.csv')

In [20]:
for index, row in df_divided_ward.iterrows():
    province = row['province']
    district = row['district']
    ward = row['ward']
    ward_point = (row['wardLat'], row['wardLon'])

    new_wards = df[(df['province']==province) & (df['district']==district) & (df['ward']==ward)]

    containing_points= []
    new_ward_points = []
    for new_ward_index, new_ward_row in new_wards.iterrows():
        new_ward_point = (new_ward_row['newWardLat'], new_ward_row['newWardLon'])
        new_ward_area_km2 = new_ward_row['newWardAreaKm2']
        new_ward_points.append(new_ward_point)

        is_contain = check_point_in_polygon(point=ward_point, polygon_center=new_ward_point, polygon_area_km2=new_ward_area_km2)
        if is_contain:
            containing_points.append(new_ward_point)
        df.loc[new_ward_index, 'isNewWardPolygonContainsWard'] = is_contain

    nearest_point = find_nearest_point(a_point=ward_point, list_of_b_points=new_ward_points)

    if len(containing_points) == 1:
        default_ward_point = containing_points[0]
    else:
        default_ward_point = nearest_point


    df.loc[(df['province']==province) & (df['district']==district) & (df['ward']==ward) & (df['newWardLat']==nearest_point[0])& (df['newWardLon']==nearest_point[1]), 'isNearestNewWard'] = True
    df.loc[(df['province']==province) & (df['district']==district) & (df['ward']==ward) & (df['newWardLat']==default_ward_point[0])& (df['newWardLon']==default_ward_point[1]), 'isDefaultNewWard'] = True

In [24]:
df

,districtCode,provinceCode,districtType,district,province,wardCode,wardType,ward,newProvinceCode,newProvince,...,newWardLon,newWardLat,newWardAreaKm2,truocsapnhap,keyNewProvinceNewWard,count_unit,isDividedWard,isNewWardPolygonContainsWard,isNearestNewWard,isDefaultNewWard
0,1,1,Quận,Quận Ba Đình,Thành phố Hà Nội,1.0,Phường,Phường Phúc Xá,1,Thủ đô Hà Nội,...,105.845,21.05670,15.09,Phường Phúc Xá,Thủ đô Hà Nội_Hồng Hà,1.0,False,NaN,NaN,NaN
1,1,1,Quận,Quận Ba Đình,Thành phố Hà Nội,4.0,Phường,Phường Trúc Bạch,1,Thủ đô Hà Nội,...,105.838,21.03860,2.97,Phường Trúc Bạch,Thủ đô Hà Nội_Ba Đình,1.0,False,NaN,NaN,NaN
2,1,1,Quận,Quận Ba Đình,Thành phố Hà Nội,6.0,Phường,Phường Vĩnh Phúc,1,Thủ đô Hà Nội,...,105.816,21.03810,2.68,Phường Vĩnh Phúc,Thủ đô Hà Nội_Ngọc Hà,1.0,False,NaN,NaN,NaN
3,1,1,Quận,Quận Ba Đình,Thành phố Hà Nội,7.0,Phường,Phường Cống Vị,1,Thủ đô Hà Nội,...,105.814,21.02750,2.60,Phường Cống Vị,Thủ đô Hà Nội_Giảng Võ,2.0,True,False,NaN,NaN
4,1,1,Quận,Quận Ba Đình,Thành phố Hà Nội,7.0,Phường,Phường Cống Vị,1,Thủ đô Hà Nội,...,105.816,21.03810,2.68,Phường Cống Vị,Thủ đô Hà Nội_Ngọc Hà,2.0,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10821,973,96,Huyện,Huyện Ngọc Hiển,Tỉnh Cà Mau,32242.0,Xã,Xã Viên An,34,Tỉnh Cà Mau,...,104.820,8.63765,271.20,Xã Viên An,Tỉnh Cà Mau_Đất Mũi,2.0,True,True,True,True
10822,973,96,Huyện,Huyện Ngọc Hiển,Tỉnh Cà Mau,32244.0,Thị trấn,Thị trấn Rạch Gốc,34,Tỉnh Cà Mau,...,104.943,8.64616,237.70,Thị trấn Rạch Gốc,Tỉnh Cà Mau_Phan Ngọc Hiển,1.0,False,NaN,NaN,NaN
10823,973,96,Huyện,Huyện Ngọc Hiển,Tỉnh Cà Mau,32245.0,Xã,Xã Tân Ân,34,Tỉnh Cà Mau,...,104.820,8.63765,271.20,Xã Tân Ân,Tỉnh Cà Mau_Đất Mũi,2.0,True,False,NaN,NaN
10824,973,96,Huyện,Huyện Ngọc Hiển,Tỉnh Cà Mau,32245.0,Xã,Xã Tân Ân,34,Tỉnh Cà Mau,...,104.943,8.64616,237.70,Xã Tân Ân,Tỉnh Cà Mau_Phan Ngọc Hiển,2.0,True,False,True,True
